# 2. 의사결정요소 Feature engineering

## 2.1. 의사결정요소 1 : 밀집도
### 생산가능인구수
- 국토정보플랫폼 국토정보맵 국토통계지도(http://map.ngii.go.kr/ms/map/NlipMap.do)의 격자별 생산가능인구수 데이터(250X250) 사용 

### 자동차보유*전기차비율
- 제공된 데이터는 100X100 격자 기준이나, 타 요소와의 격자 통일을 위해 처리함(250X250)
- Null 값은 존재하지 않으나 0값이 대부분을 차지하고 있어, 데이터 변환의 수월성을 위해 0값을 제거함
- 전기차보급현황 데이터를 통해 계산한 행정구역별 전기차 보급 비율을 자동차등록현황에 곱해줌

### 토지이용압축도
- 격자 내 건축물 연면적 합 ÷ 격자 면적 × 100

### 라이브러리 및 데이터 불러오기

In [1]:
from geoband.API import *

In [2]:
import folium
import json
import geopandas as gpd

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import haversine

from shapely.geometry import Point, Polygon, MultiPolygon

import platform

from IPython.display import set_matplotlib_formats

if platform.system() == 'Darwin':
    plt.rc('font', family = 'AppleGothic')
elif platform.system() == 'Windows':
    plt.rc('font', family = 'Malgun Gothic')

plt.rc('axes', unicode_minus=False)

set_matplotlib_formats('retina')

import warnings
warnings.filterwarnings(action='ignore')

import pathlib
from tqdm import tqdm, notebook
import os

%matplotlib inline

/opt/app-root/lib/python3.6/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
# 데이터 불러오기
input_path = pathlib.Path('./data')
if not input_path.is_dir():
    input_path.mkdir()


GetCompasData('SBJ_2009_001', '3', input_path.joinpath('03.광양시_자동차등록현황_격자(100X100).geojson'))
GetCompasData('SBJ_2009_001', '6', input_path.joinpath('06.광양시_전기차보급현황(연도별,읍면동별).csv'))
GetCompasData('SBJ_2009_001', '15', input_path.joinpath('15.광양시_건물정보.geojson'))
GetCompasData('SBJ_2009_001', '20', input_path.joinpath('20.광양시_행정경계(읍면동).geojson'))

# 국토정보맵 데이터
생산가능인구수 = gpd.read_file('./data/생산가능인구수/vl_blk.shp', encoding = 'EUC-KR')

# COMPAS 데이터
자동차등록현황 = gpd.read_file('./data/03.광양시_자동차등록현황_격자(100X100).geojson')
전기차보급현황 = pd.read_csv('./data/06.광양시_전기차보급현황(연도별,읍면동별).csv')
건물정보 = gpd.read_file('./data/15.광양시_건물정보.geojson')
행정경계_읍면동 = gpd.read_file('./data/20.광양시_행정경계(읍면동).geojson')

### 생산가능인구수

In [4]:
# 생산가능인구수의 좌표계를 EPSG:4326으로 변환함
생산가능인구수 = 생산가능인구수.to_crs({'init': 'epsg:4326'})

# feature_gdf 데이터 프레임에 밀집도 feature 취합함
feature_gdf = 생산가능인구수.drop('lbl', axis = 1).rename(columns = {'val':'생산가능인구수'}).copy()

### 자동차보유*전기차비율

In [5]:
# 자동차등록현황 250X250 격자화 및 feature_gdf에 취합
feature_gdf['자동차등록현황'] = 0
자동차등록현황 = 자동차등록현황[자동차등록현황['totale'] != 0]

for i in tqdm(자동차등록현황.index):
    for j in feature_gdf.index:
        if 자동차등록현황['geometry'][i].intersects(feature_gdf['geometry'][j]):
            feature_gdf['자동차등록현황'][j] += 자동차등록현황['totale'][i]

100%|██████████| 2626/2626 [18:16<00:00,  2.40it/s]


- 자동차등록현황에 광양시 전기차 비율 반영

In [6]:
# 전기차보급현황의 행정경계 표시하기 위해 dataframe merge
전기차보급현황 = 전기차보급현황.groupby('행정구역')[['보급현황']].sum()
전기차보급현황 = pd.merge(행정경계_읍면동, 전기차보급현황, left_on = 'ADM_DR_NM', right_on = '행정구역')
전기차보급현황 = 전기차보급현황.rename(columns = {'ADM_DR_NM' : '행정구역'})[['행정구역', '보급현황', 'geometry']]

In [7]:
%%time
ft2 = feature_gdf.copy()
ft2['행정구역'] = 'NaN'

# 격자에 데이터 넣기
for i in range(len(ft2)):
    point = ft2['geometry'][i].centroid
    for j in range(len(전기차보급현황)):
        if (point.within(전기차보급현황['geometry'][j])):           #격자의 centroid가 행정경계 안에 있으면 행정구역명 넣기
            ft2['행정구역'][i] = 전기차보급현황['행정구역'][j]
            break
        elif (ft2['geometry'][i].intersects(전기차보급현황['geometry'][j])):     #경계에 걸쳐있어 null값으로 처리되는 데이터는 intersects로 겹치면 행정구역명 넣기
            ft2['행정구역'][i] = 전기차보급현황['행정구역'][j]
            break

CPU times: user 19.2 s, sys: 0 ns, total: 19.2 s
Wall time: 20.2 s


In [8]:
# 처리되지 않는 null값은 따로 처리(3건)
ft2['행정구역'][1212] = '광양읍'
ft2['행정구역'][1213] = '광양읍'
ft2['행정구역'][7606] = '태인동'

In [9]:
# 전기차보급현황 dataframe에 행정구역별 전기차 보급 비율 계산해서 추가
df = ft2.groupby('행정구역')[['자동차등록현황']].sum()
df2 = pd.merge(df, 전기차보급현황, on = '행정구역')
df2['보급비율'] = df2['보급현황']/df2['자동차등록현황']
df2

,행정구역,자동차등록현황,보급현황,geometry,보급비율
0,골약동,778,4,"MULTIPOLYGON (((127.68056 34.92848, 127.68105 ...",0.005141
1,광양읍,8241,102,"MULTIPOLYGON (((127.59070 34.91870, 127.59150 ...",0.012377
2,광영동,1441,12,"MULTIPOLYGON (((127.71676 34.96939, 127.71691 ...",0.008328
3,금호동,2719,23,"MULTIPOLYGON (((127.73203 34.95128, 127.73243 ...",0.008459
4,다압면,311,0,"MULTIPOLYGON (((127.64117 35.16850, 127.64140 ...",0.000000
5,봉강면,542,4,"MULTIPOLYGON (((127.58037 35.10902, 127.58052 ...",0.007380
6,옥곡면,1091,8,"MULTIPOLYGON (((127.64878 35.03547, 127.64902 ...",0.007333
7,옥룡면,534,11,"MULTIPOLYGON (((127.59872 35.12006, 127.59919 ...",0.020599
8,중마동,10658,78,"MULTIPOLYGON (((127.70376 34.96198, 127.70542 ...",0.007318
9,진상면,492,5,"MULTIPOLYGON (((127.65639 35.11680, 127.65698 ...",0.010163


In [10]:
%%time
ft2['자동차보유*전기차비율'] = ft2['자동차등록현황'].astype(float)

for i in range(len(ft2)):
    for j in range(len(df2)):
        if (ft2['행정구역'][i] == df2['행정구역'][j]):
            ft2['자동차보유*전기차비율'][i] =  ft2['자동차보유*전기차비율'][i]*df2['보급비율'][j]
            break

# feature_gdf에 취합
feature_gdf['자동차보유*전기차비율'] = ft2['자동차보유*전기차비율']
feature_gdf.drop('자동차등록현황', axis=1, inplace=True)

CPU times: user 2.25 s, sys: 11.9 ms, total: 2.26 s
Wall time: 2.39 s


### 토지이용압축도

In [11]:
# 건물정보 데이터를 활용하여 격자 내 연면적 합 구하기
건물정보_연면적 = 건물정보[['건물연면적','geometry']]

feature_gdf['연면적'] = 0

for i in tqdm(건물정보_연면적.index):
    point = 건물정보_연면적['geometry'][i].centroid
    for j in feature_gdf.index:
        if(point.within(feature_gdf['geometry'][j])):
            feature_gdf['연면적'][j] += 건물정보_연면적['건물연면적'][i]

100%|██████████| 23556/23556 [1:31:51<00:00,  4.27it/s]


In [12]:
# 토지이용압축도 구하기 (격자 내 건축물 연면적 합 ÷ 격자 면적 × 100)
feature_gdf['토지이용압축도'] = feature_gdf['연면적']/62500 * 100
feature_gdf.drop('연면적', axis=1, inplace=True)

In [13]:
# feature_gdf 저장
feature_gdf.to_file('./data/feature_gdf.geojson', driver = 'GeoJSON')